<a href="https://colab.research.google.com/github/8Mateo8/caracoles_IA/blob/main/Proyecto_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf

ruta_dataset = '/content/drive/MyDrive/imagenes_IA'

IMG_SIZE = (600, 600)

BATCH_SIZE = 8

SEED = 42

print("Cargando set de ENTRENAMIENTO...")
train_ds = tf.keras.utils.image_dataset_from_directory(
    ruta_dataset,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

print("Cargando set de VALIDACIÓN...")
val_ds = tf.keras.utils.image_dataset_from_directory(
    ruta_dataset,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

class_names = train_ds.class_names
print(f"\nClases encontradas: {class_names}")

Cargando set de ENTRENAMIENTO...
Found 138 files belonging to 3 classes.
Using 111 files for training.
Cargando set de VALIDACIÓN...
Found 138 files belonging to 3 classes.
Using 27 files for validation.

Clases encontradas: ['Burn', 'Skidmark', 'Turbo']


In [ ]:
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

base_model = EfficientNetB7(
    weights='imagenet',
    include_top=False,
    input_shape=(600, 600, 3)
)

base_model.trainable = False

model = models.Sequential([
    layers.Input(shape=(600, 600, 3)),
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.Lambda(preprocess_input),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

258076736/258076736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=150,
    callbacks=[early_stopping]
)


Epoch 1/150
8/8 ━━━━━━━━━━━━━━━━━━━━ 384s 42s/step - accuracy: 0.3180 - loss: 1.1603 - val_accuracy: 0.2000 - val_loss: 1.2157
Epoch 2/150
8/8 ━━━━━━━━━━━━━━━━━━━━ 320s 40s/step - accuracy: 0.3131 - loss: 1.1517 - val_accuracy: 0.2000 - val_loss: 1.2094
Epoch 3/150
8/8 ━━━━━━━━━━━━━━━━━━━━ 291s 37s/step - accuracy: 0.4091 - loss: 1.1312 - val_accuracy: 0.2667 - val_loss: 1.2033
Epoch 4/150
8/8 ━━━━━━━━━━━━━━━━━━━━ 320s 37s/step - accuracy: 0.3763 - loss: 1.1120 - val_accuracy: 0.3333 - val_loss: 1.1983
Epoch 5/150
8/8 ━━━━━━━━━━━━━━━━━━━━ 300s 38s/step - accuracy: 0.4765 - loss: 0.9971 - val_accuracy: 0.4000 - val_loss: 1.1977
Epoch 6/150
8/8 ━━━━━━━━━━━━━━━━━━━━ 288s 37s/step - accuracy: 0.3817 - loss: 1.0705 - val_accuracy: 0.4000 - val_loss: 1.1925
Epoch 7/150
8/8 ━━━━━━━━━━━━━━━━━━━━ 349s 41s/step - accuracy: 0.4923 - loss: 1.0113 - val_accuracy: 0.4000 - val_loss: 1.1840
Epoch 8/150
8/8 ━━━━━━━━━━━━━━━━━━━━ 293s 37s/step - accuracy: 0.4313 - loss: 1.0586 - val_accuracy: 0.4000 - v

In [ ]:
model.save('caracoles_v4.h5')

print("Modelo guardado exitosamente.")

from google.colab import files
files.download('caracoles_v4.h5')

Modelo guardado exitosamente.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>